# Using Gridsearch on Keras Model

### TODO

1. Add all of the options

---

In [1]:
## DO THIS BEFORE IMPORTING KERAS OR TENSOR TO USE PLAIDML
import plaidml.keras
plaidml.keras.install_backend()

# Help MacOS be able to use Keras
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# Gets rid of the processor warning.
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [27]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import time
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
from pprint import pprint
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from util import databases
from util import dataloader
from util import grid_models
from util.reddit_functions import Labeler
from util.reddit_functions import plot_confusion_matrix
from util.grid_models import custom_stop_words

In [4]:
def create_model():

    model = Sequential()
    model.add(Dense(8, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(1, activation='sigmoid'))

    adam = Adam(lr=.01)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [5]:
def score_model(model):
    
    print(f'Train Score: {model.score(X_train, y_train)}')
    print(f'Test Score: {model.score(X_test, y_test)}')
    print(f'AUC Score: {roc_auc_score(y_test, y_proba, multi_class="ovr")}')
    print(classification_report(y_test, y_pred, digits=3))

In [6]:
# subreddit_list = ['css', 'html', 'javascript', 'php', 'perl', 'java', 'datascience', 'machinelearning', 'etl', 'python', 'dataengineering']

In [39]:
subreddit_list = ['datascience','machinelearning']

In [40]:
df = dataloader.data_selector(subreddit_list, 'sqlite')

Connection to SQLite DB successful


In [41]:
df.sample(10)

,title,subreddit,date
22013,[News] The Roundabout Drone Dataset (rounD) – ...,machinelearning,2020-04-28
17685,ML project on speech,datascience,2020-04-25
8173,PhD hunger at medium/low-tier companies,datascience,2020-04-20
10390,[D] Anyone have experience using Compressed Se...,machinelearning,2020-04-21
19430,Scaling data before prediction,datascience,2020-04-27
14458,[Discussion] Paper: Wrong results on the imple...,machinelearning,2020-04-23
8950,[D] Looking for a Pytorch implementation of Re...,machinelearning,2020-04-20
13353,State of the art techniques to get the most im...,datascience,2020-04-23
7675,White House Announces New Partnership to Unlea...,datascience,2020-04-20
9721,How would you preprocess features like these?,datascience,2020-04-21


In [45]:
X = df['title']
y = df['subreddit']

In [46]:
labeler = Labeler()
labeler.fit(y)
y = labeler.transform(y)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

In [48]:
tfidf = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(1,2))
X_train = tfidf.fit_transform(X_train).toarray()
X_test = tfidf.transform(X_test).toarray()

In [49]:

model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 10)                165160    
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 165,171
Trainable params: 165,171
Non-trainable params: 0
_________________________________________________________________


In [50]:
# model = create_model()
model.fit(X_train, y_train, epochs=3, batch_size=128)

Train on 16821 samples
Epoch 1/3
16821/16821 [==============================] - 1s 76us/sample - loss: 0.6031 - accuracy: 0.8091
Epoch 2/3
16821/16821 [==============================] - 1s 66us/sample - loss: 0.3662 - accuracy: 0.9829
Epoch 3/3
16821/16821 [==============================] - 1s 66us/sample - loss: 0.2005 - accuracy: 0.9903


In [18]:
model.evaluate(X_test, y_test)

14340/14340 [==============================] - 4s 278us/sample - loss: -291.1783 - accuracy: 0.1916


[-291.17831792046667, 0.19156207]

In [19]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)
score_model(model)
plot_confusion_matrix(model, y_true=y_test, y_pred=y_pred, classes=labeler.classes_)

AttributeError: 'Sequential' object has no attribute 'score'

In [ ]:
fitted_models